# GRIB to XArray

The purpose of this notebook is to read a bunch of GRIB files with PyGrib, and directly build an XArray dataset.
The motivation is to avoid having an intermediary file format like NetCDF, and consequently save a lot on IO.

It would be nice to do it using the cfgrib engine for XArray, but cfgrib makes it impractical to open all the fields we want, because it doesn't allow us to open multiple fields with different vertical levels.

In [ ]:
%load_ext autoreload
%autoreload 2

The overall strategy is to 

1. Read and filter an GRIB file
2. Put the fields we want in an XArray Dataset
3. Read many grib files in parallel.
4. Merge the XArray datasets from different files.

## 1 Read and filter a grib file

In [ ]:
import datetime
import multiprocessing
import numpy as np
import os
import pathlib
import pandas as pd
import pygrib
from tqdm.notebook import tqdm
import xarray as xr

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))
GRIB_INPUT_DIR = DATA_DIR / 'data/gdps/2020020112'

In [ ]:
def from_short_name(message):
    to_extract = [
        'al',
        'hpbl',
        'prate',
        'prmsl',
        'thick',
        '10si',
        '10wdir',
        '10u',
        '10v',
        '2d',
        '2r',
        '2t',
    ]
    if message.shortName in to_extract:
        return message.shortName
    else:
        return False

In [ ]:
class ShortNameLevelExtractor:
    def __init__(self, name, levels):
        self.name = name
        self.levels = levels
        
    def __call__(self, message):
        if message.shortName == self.name:
            for level in self.levels:
                if message.level == level:
                    return f'{self.name}_{level}'
        return False

In [ ]:
class CompositeExtractor:
    def __init__(self, extractors):
        self.extractors = extractors
        
    def __call__(self, message):
        for e in self.extractors:
            is_included = e(message)
            if is_included:
                return is_included
        return False

In [ ]:
extractor = CompositeExtractor([
    from_short_name,
    ShortNameLevelExtractor('t', [850, 500]),
    ShortNameLevelExtractor('gh', [1000, 850, 500]),
    ShortNameLevelExtractor('q', [850, 500]),
    ShortNameLevelExtractor('u', [500]),
    ShortNameLevelExtractor('v', [500]),
])

In [ ]:
def extract_fields(grib_iter, extractor):
    fields = {}
    for message in grib_iter:
        label = extractor(message)
        if label:
            fields[label] = message
            
    return fields

In [ ]:
def do_one_file(grib_file_path, extractor):
    print(grib_file_path)
    gribfile = pygrib.open(str(grib_file_path))
    dataset = file_to_xarray(gribfile, extractor)
    gribfile.close()
    
    return dataset

In [ ]:
def pass_to_xarray(pass_dir, extractor):
    input_path = pathlib.Path(GRIB_INPUT_DIR)
    input_files = sorted(list(input_path.glob('*.grib2')))
    
    input_files = input_files[:8]
    
    with multiprocessing.Pool() as pool:
        datasets = pool.starmap(do_one_file, [(f, extractor) for f in input_files[:10]])
    
    return xr.concat(datasets, dim='step')

In [ ]:
def file_to_xarray(grib_file, extractor):
    sample = next(iter(grib_file))
    
    lats, lons = sample.latlons()
    lats, lons = lats[:,0], lons[0]
    
    datetime = grib_date_to_pandas(sample.dataDate, sample.dataTime)
    datetime = [datetime]
    
    if sample.stepUnits == 1:
        step = [pd.Timedelta(sample.step, 'h')]
    else:
        raise ValueError('Unhandled step units')
        
    fields = {}
    for msg in grib_file:
        label = extractor(msg)
        if label:
            data_array = message_to_xarray(msg, lats, lons, step, datetime)
            fields[label] = data_array
            
    return xr.Dataset(fields)

In [ ]:
def message_to_xarray(grib_message, lats, lons, step, datetime):
    values = np.array(grib_message.values, dtype=np.float32)
    values = np.expand_dims(values, axis=0)
    values = np.expand_dims(values, axis=0)

    da = xr.DataArray(
        values, dims=['datetime', 'step', 'lat', 'lon'], 
        coords={'lat': lats, 'lon': lons, 'datetime': datetime, 'step': step}
    )
    
    da.attrs['units'] = grib_message.units
    
    return da

In [ ]:
def grib_date_to_pandas(date, time):
    date, time = str(date), str(time)
    date_string = f'{date[:4]}-{date[4:6]}-{date[6:8]} {time[:2]}:{time[2:4]}'
    return pd.Timestamp(date_string)

In [ ]:
big_dataset = pass_to_xarray(GRIB_INPUT_DIR, extractor)

In [ ]:
big_dataset

In [ ]:
big_dataset = big_dataset.assign_coords(valid=lambda x: x.datetime + x.step)

In [ ]:
input_path = pathlib.Path(GRIB_INPUT_DIR)
input_files = sorted(list(input_path.glob('*.grib2')))

datasets = []

for f in input_files[8:12]:
    gribfile = pygrib.open(str(f))
    dataset = file_to_xarray(gribfile, extractor)
    datasets.append(dataset)

In [ ]:
d = xr.merge(datasets)

In [ ]:
d

In [ ]:
d.isel(step=0)['2t'].plot()

In [ ]:
d.isel(step=1)['thick'].plot()

In [ ]:
def sizeof_dataset(dataset):
    total = 0
    for var in dataset.variables:
        array = dataset[var]
        total += array.size * array.dtype.itemsize
        
    return total

In [ ]:
sizeof_dataset(d) / 1024**2 

In [ ]:
d.sizes

In [ ]:
gribfile = pygrib.open(str(input_files[0]))

In [ ]:
dataset = xr.Dataset(data_vars=data_arrays)

In [ ]:
dataset

## 3. Read many grib files in parallel. 

In [ ]:
import os
import multiprocessing

In [ ]:
os.getenv('SLURM_CPUS_PER_TASK')

In [ ]:
len(input_files)

In [ ]:
def do_one_file(path):
    gribfile = pygrib.open(str(path))
    return file_to_xarray(gribfile, extractor)

In [ ]:
in_list = input_files[:20]

with multiprocessing.Pool(int(4)) as pool:
    datasets = list(tqdm(pool.imap(do_one_file, in_list), total=len(in_list)))

## 4. Interpolate at stations

see notebooks 1901 1902


In [ ]:
import pymongo

from smc01.interpolate.obs import MongoIEMDatabase

In [ ]:
MONGO_URL = 'localhost'
MONGO_PORT = 27017
USERNAME = None
PASSWORD = None
ADMIN_DB = 'admin'
COLLECTION = 'iem'
DB = 'smc01_raw_obs'

In [ ]:
client = pymongo.MongoClient(
    host=MONGO_URL,
    port=MONGO_PORT,
    tz_aware=True,
    authSource=ADMIN_DB,
    username=USERNAME,
    password=PASSWORD,
)

In [ ]:
db = MongoIEMDatabase(client, DB, COLLECTION)

In [ ]:
begin_date = big_dataset.valid.min().data.item()
begin_date = datetime.datetime.utcfromtimestamp(begin_date // 1e9)

end_date = big_dataset.valid.max().data.item()
end_date = datetime.datetime.utcfromtimestamp(end_date // 1e9)

In [ ]:
station_info = db.station_info()

In [ ]:
station_info

In [ ]:
at_stations = big_dataset.interp(
    {
        "lat": xr.DataArray(station_info["lat"], dims="station"),
        "lon": xr.DataArray(station_info["lon"], dims="station"),
    }
)

In [ ]:
big_dataset

In [ ]:
at_stations

In [ ]:
at_stations.station

In [ ]:
at_stations = at_stations.assign_coords(
    station=xr.DataArray(station_info["station"], dims="station")
)

In [ ]:
at_stations.station[0].item()

In [ ]:
at_stations

In [ ]:
by_valid = {
    valid_time: group for valid_time, group in at_stations.groupby("valid")
}

In [ ]:
one_valid_key = next(iter(by_valid))

In [ ]:
one_valid = by_valid[one_valid_key]

In [ ]:
one_valid

In [ ]:
one_step = one_valid.isel(stacked_datetime_step=0)

In [ ]:
date, step = one_step.stacked_datetime_step.item()

In [ ]:
type(date)

In [ ]:
step

In [ ]:
step.total_seconds() / 3600

In [ ]:
date = pd.to_datetime(date, unit="s")
step = datetime.timedelta(hours=step.total_seconds() / 3600)

In [ ]:
date

In [ ]:
step

In [ ]:
date

In [ ]:
date.to_pydatetime()

In [ ]:
big_dataset

In [ ]:
big_dataset.data_vars.keys()

In [ ]:
for v in big_dataset.data_vars.keys():
    print(v)